# Compare and annotation between methods

In [ ]:
# Seurat 
library(Seurat)

# Statistics
library(caret)

# Data 
library(dplyr)

# Plotting
library(ggplot2)
library(RColorBrewer)
library(patchwork)

In [ ]:
# Set working directory to project root
setwd("/research/peer/fdeckert/FD20200109SPLENO")

In [ ]:
# Source files
source("plotting_global.R")

# Parameter settings

In [ ]:
so_file <- "data/object/seurat.rds"

# Plotting Theme
ggplot2::theme_set(theme_global_set()) 

# Load objects

In [ ]:
so <- readRDS(so_file)

In [ ]:
# Meta data 
scvibatch_meta <- read.csv("data/object/components/meta_data/scvibatch_meta.csv", sep = ",", row.names = 1, header = TRUE)

In [ ]:
so$seurat_clusters <- scvibatch_meta$`scvi_snn_res.0.8`

# Compare clustering 

In [ ]:
cluster_tissue <- ggplot(so@meta.data, aes(x = seurat_clusters, fill = tissue)) + 
  geom_bar(stat = "count", position = "fill") + 
  scale_fill_manual(values = so_color$tissue) +
  ggtitle("Cluster frequency") + xlab("Cluster") + ylab("Cell frequency") + 
  theme(legend.position = "bottom")

cluster_treatment <- ggplot(so@meta.data, aes(x = seurat_clusters, fill = treatment)) + 
  geom_bar(stat = "count", position = "fill") + 
  scale_fill_manual(values = so_color$treatment) +
  ggtitle("Cluster frequency") + xlab("Cluster") + ylab("Cell frequency") + 
  theme(legend.position = "bottom")

options(repr.plot.width = 10, repr.plot.height = 5)
cluster_tissue + cluster_treatment + plot_layout(ncol = 2)
ggsave(cluster_tissue, filename = "result/plot/seurat/cluster_tissue_scvibatch.png", width = 6, height = 3)
ggsave(cluster_treatment, filename = "result/plot/seurat/cluster_treatment_scvibatch.png", width = 6, height = 3)

# Import scanvi object

In [ ]:
library(reticulate)
sc <- import("scipy", convert = FALSE)

# Normalized data matrix 
scanvi_prog_cpg_data <- sc$sparse$load_npz("data/object/components/slots/scanvi_prog_cpg_data.npz") %>%  py_to_r() %>% as.matrix() %>% t()
colnames(scanvi_prog_cpg_data) <- read.csv("data/object/components/slots/scanvi_prog_cpg_data_cellid.txt")[, 1]
rownames(scanvi_prog_cpg_data) <- read.csv("data/object/components/slots/scanvi_prog_cpg_data_genes.txt")[, 1]

library(reticulate)
np <- import("numpy", convert = FALSE)

# Latent space representation
scanvi_prog_cpg_latend <- np$loadtxt("data/object/components/reductions/scanvi_prog_cpg_latent.txt") %>% py_to_r()
rownames(scanvi_prog_cpg_latend) <- read.csv("data/object/components/reductions/scanvi_prog_cpg_latent_cellid.txt")[, 1]

scanvi_prog_cpg_meta <- read.csv("data/object/components/meta_data/scanvi_prog_cpg_meta.csv", row.names = 1)

# Create Seurat object 
scanvi_prog_cpg_scvi <- CreateSeuratObject(counts = GetAssayData(object = so[, colnames(so) %in% colnames(scanvi_prog_cpg_data)]), assay = "SCANVI")
scanvi_prog_cpg_scvi <- AddMetaData(scanvi_prog_cpg_scvi, scanvi_prog_cpg_meta)
scanvi_prog_cpg_scvi@assays$SCANVI@data <- as(scanvi_prog_cpg_data, "dgCMatrix")
scanvi_prog_cpg_scvi[["scanvi_latent"]] <- CreateDimReducObject(embeddings = as.matrix(scanvi_prog_cpg_latend), key = "scanvi_latent_", assay = "SCANVI")

In [ ]:
library(reticulate)
sc <- import("scipy", convert = FALSE)

# Normalized data matrix 
scanvi_prog_nacl_data <- sc$sparse$load_npz("data/object/components/slots/scanvi_prog_nacl_data.npz") %>%  py_to_r() %>% as.matrix() %>% t()
colnames(scanvi_prog_nacl_data) <- read.csv("data/object/components/slots/scanvi_prog_nacl_data_cellid.txt")[, 1]
rownames(scanvi_prog_nacl_data) <- read.csv("data/object/components/slots/scanvi_prog_nacl_data_genes.txt")[, 1]

library(reticulate)
np <- import("numpy", convert = FALSE)

# Latent space representation
scanvi_prog_nacl_latend <- np$loadtxt("data/object/components/reductions/scanvi_prog_nacl_latent.txt") %>% py_to_r()
rownames(scanvi_prog_nacl_latend) <- read.csv("data/object/components/reductions/scanvi_prog_nacl_latent_cellid.txt")[, 1]

scanvi_prog_nacl_meta <- read.csv("data/object/components/meta_data/scanvi_prog_nacl_meta.csv", row.names = 1)

# Create Seurat object 
scanvi_prog_nacl_scvi <- CreateSeuratObject(counts = GetAssayData(object = so[, colnames(so) %in% colnames(scanvi_prog_nacl_data)]), assay = "SCANVI")
scanvi_prog_nacl_scvi <- AddMetaData(scanvi_prog_nacl_scvi, scanvi_prog_nacl_meta)
scanvi_prog_nacl_scvi@assays$SCANVI@data <- as(scanvi_prog_nacl_data, "dgCMatrix")
scanvi_prog_nacl_scvi[["scanvi_latent"]] <- CreateDimReducObject(embeddings = as.matrix(scanvi_prog_nacl_latend), key = "scanvi_latent_", assay = "SCANVI")

# Dim reduction and clustering 

In [ ]:
source("bin/seurat_function.R")

prefix <- tolower("SCANVI")
name_nn   <- paste0(prefix, "_nn")
name_snn  <- paste0(prefix, "_snn")
name_umap <- paste0(prefix, "_umap")
name_nno  <- paste0(prefix, "_nno")
cluster_res  = 0.8

In [ ]:
# scanvi_prog_cpg_scvi <- FindNeighbors(
#         object          = scanvi_prog_cpg_scvi,
#         assay           = "SCANVI",
#         dims            = 1:30,
#         k.param         = 20,
#         reduction       = "scanvi_latent",
#         return.neighbor = TRUE,
#         graph.name      = name_nno
#     )

# scanvi_prog_cpg_scvi <- FindNeighbors(
#         object          = scanvi_prog_cpg_scvi,
#         assay           = "SCANVI",
#         dims            = 1:30,
#         k.param         = 20,
#         reduction       = "scanvi_latent",
#         graph.name      = c(name_nn, name_snn)
#     )

# scanvi_prog_cpg_scvi <- FindClusters(
#     object         = scanvi_prog_cpg_scvi,
#     resolution     = cluster_res,
#     graph.name     = name_snn
# )

# scanvi_prog_cpg_scvi <- RunUMAP(
#         object         = scanvi_prog_cpg_scvi,
#         assay          = "SCANVI",
#         n.neighbors    = 20,
#         nn.name        = name_nno,
#         reduction.name = paste0(name_umap, "_nno")
# )

reduction <- "scanvi_umap_nno"
dplot_1 <- DimPlot(scanvi_prog_cpg_scvi, reduction = reduction, group.by = "scanvi_snn_res.0.8", label = TRUE) & 
    theme(aspect.ratio = 1, legend.position = "none")
dplot_2 <- DimPlot(scanvi_prog_cpg_scvi, reduction = reduction, group.by = "fine_labels", label = FALSE) & 
    theme(aspect.ratio = 1, legend.position = "bottom") & 
    guides(color = guide_legend(ncol = 3, override.aes = list(size = 3)))

options(repr.plot.width = 10, repr.plot.height = 10)
dplot <- dplot_1 + dplot_2 + plot_layout(ncol = 2)
dplot
ggsave(dplot, filename = "result/plot/seurat/dimplot_scanvi_cpg_1.png", width = 9, height = 9)

In [ ]:
# scanvi_prog_nacl_scvi <- FindNeighbors(
#         object          = scanvi_prog_nacl_scvi,
#         assay           = "SCANVI",
#         dims            = 1:30,
#         k.param         = 20,
#         reduction       = "scanvi_latent",
#         return.neighbor = TRUE,
#         graph.name      = name_nno
#     )

# scanvi_prog_nacl_scvi <- FindNeighbors(
#         object          = scanvi_prog_nacl_scvi,
#         assay           = "SCANVI",
#         dims            = 1:30,
#         k.param         = 20,
#         reduction       = "scanvi_latent",
#         graph.name      = c(name_nn, name_snn)
#     )

# scanvi_prog_nacl_scvi <- FindClusters(
#     object         = scanvi_prog_nacl_scvi,
#     resolution     = cluster_res,
#     graph.name     = name_snn
# )

# scanvi_prog_nacl_scvi <- RunUMAP(
#         object         = scanvi_prog_nacl_scvi,
#         assay          = "SCANVI",
#         n.neighbors    = 20,
#         nn.name        = name_nno,
#         reduction.name = paste0(name_umap, "_nno")
# )

reduction <- "scanvi_umap_nno"
dplot_1 <- DimPlot(scanvi_prog_nacl_scvi, reduction = reduction, group.by = "scanvi_snn_res.0.8", label = TRUE) & 
    theme(aspect.ratio = 1, legend.position = "none")
dplot_2 <- DimPlot(scanvi_prog_nacl_scvi, reduction = reduction, group.by = "C_scANVI", label = FALSE) & 
    theme(aspect.ratio = 1, legend.position = "bottom") & 
    guides(color = guide_legend(ncol = 3, override.aes = list(size = 3)))

options(repr.plot.width = 10, repr.plot.height = 10)
dplot <- dplot_1 + dplot_2 + plot_layout(ncol = 2)
dplot
ggsave(dplot, filename = "result/plot/seurat/dimplot_scanvi_nacl_1.png", width = 9, height = 9)

fplot_1 <- FeaturePlot(scanvi_prog_nacl_scvi, reduction = reduction, features = "fine_delta_score_p") & theme(aspect.ratio = 1) & ggtitle("SingleR score")
fplot_2 <- FeaturePlot(scanvi_prog_nacl_scvi, reduction = reduction, features = "p_score_scANVI") & theme(aspect.ratio = 1) & ggtitle("SCANVI score")

fplot <- dplot_1 + fplot_1 + fplot_2 + plot_layout(ncol = 3)
options(repr.plot.width = 15, repr.plot.height = 5)
fplot
# ggsave(fplot_tcr_cd3, filename = "result/plot/seurat/fplot_tcr_cd3.png", width = 6, height = 3)

In [ ]:
fine_label_freq <- ggplot(rbind(scanvi_prog_nacl_scvi@meta.data, scanvi_prog_cpg_scvi@meta.data), aes(x = fine_labels, fill = treatment)) + 
  geom_bar(stat = "count", position = "dodge") + 
  scale_fill_manual(values = so_color$treatment) +
  ggtitle("Label frequency") + xlab("Fine label") + ylab("Label frequency") + 
  theme(legend.position = "bottom")
fine_label_freq